---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevin-crotty.com)
    
Jones Graduate School of Business, Rice University

---


# PULL DATA

In [1]:
import pandas as pd
from pandas_datareader import DataReader as pdr

# Pull data from FRED
files = ["DFII" + x for x in ["5", "7", "10", "20", "30"]]
df = pdr(files, "fred", start=1920) / 100
df.index.name = "date"
df = df.reset_index()

# Keep 1st observations of each month
df["month"] = df.date.dt.to_period("M").astype(str)
df = df.groupby("month").first()

# Clean-up data
df = df.drop(columns=["date"])
df = df.dropna(subset=["DFII20"])
df.columns = [5, 7, 10, 20, 30]
df = df.stack()
df = df.reset_index()
df.columns = ["month", "term", "rate"]
df.head()

,month,term,rate
0,2004-07,5,0.0130
1,2004-07,7,0.0170
2,2004-07,10,0.0204
3,2004-07,20,0.0249
4,2004-08,5,0.0126


# FIGURE

In [2]:
import plotly.express as px
hover_data = dict(month=False, term=True, rate=True)
fig = px.line(
    df,
    x="term",
    y="rate",
    animation_frame="month",
    hover_name="month",
    hover_data=hover_data,
)
fig.layout.xaxis["title"] = "Years to Maturity"
fig.layout.yaxis["title"] = "Yield"
fig.update_yaxes(tickformat=".1%", range=[-0.02, df.rate.max() + 0.001])
fig.update_xaxes(tickvals=[5, 7, 10, 20, 30], range=[0, 30])
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 200
fig.show()